In [2]:
#import libraries
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
# define function to check take off flaps and decel approach
def takeoff_flap(aircraft, flaps):
    if aircraft == 'A359':
        if flaps > 16.5:
            return False
        else:
            return True
    else:
        if flaps > 5:
            return False
        else:
            return True

def decel_app(alt):
    if alt > 2000:
        return False
    else:
        return True

def b787_type(call_sign, aircraft_type):
    reduced_callsign = call_sign[3:5]
    if reduced_callsign == 'TQ':
        final_type = 'B788'
    elif reduced_callsign == 'TW':
        final_type = 'B789'
    else:
        final_type = aircraft_type
    return final_type    

In [4]:
# formatting float number
pd.options.display.float_format = '{:,.2f}'.format

# load qar to DataFrame
qar_df = pd.read_json('.\qar\qar.json')

In [5]:
sel_cols = ['aircraft_registration',
            'qar_start_time',
            'aircraft_type',
            'departure_aerodrome',
            'one_engine_taxi_out',
            'takeoff_flaps',
            'pack_off_takeoff',
            'acceleration_height',
            'climb_sd',
            'descent_sd',
            'landing_config_height',
            'reduced_flaps_landing',
            'idle_reverse_landing_performed',
            'landing_flaps',
            'one_engine_taxi_in',
            'apu_used_during_taxi_in']
qar_df = qar_df[sel_cols]

In [6]:
# convert qar start date to date column
qar_df['date'] = qar_df.qar_start_time.apply(pd.Series)
qar_df['date'] = qar_df.date.apply(pd.to_datetime)
qar_df.drop(columns='qar_start_time', inplace=True)

C:\Users\peaju\AppData\Local\Temp\ipykernel_11844\690324268.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  qar_df['date'] = qar_df.qar_start_time.apply(pd.Series)
C:\Users\peaju\AppData\Local\Temp\ipykernel_11844\690324268.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  qar_df['date'] = qar_df.qar_start_time.apply(pd.Series)


In [7]:
# set date column as index
qar_df.set_index(qar_df.date, drop=True, inplace=True)

In [8]:
qar_df.sort_index(ascending=True, inplace=True)
# select data for year 2021 and after
qar_df_2021 = qar_df.loc['2021-01-01':]

C:\Users\peaju\AppData\Local\Temp\ipykernel_11844\2720486284.py:3: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  qar_df_2021 = qar_df.loc['2021-01-01':]


In [9]:
# Change aircraft type between B788 and B789
for i, row in qar_df_2021.iterrows():
    qar_df_2021.loc[i,'aircraft_type'] = b787_type(row[0], row[1])

In [10]:
# create new columns "is_less_flaps_takeoff"

for i, row in qar_df_2021.iterrows():
    qar_df_2021.loc[i,'is_less_flaps_takeoff'] = takeoff_flap(row[1], row[4])

In [11]:
# create new columns "is_decel_approach"

for i, col in qar_df_2021.iterrows():
    qar_df_2021.loc[i,'is_decel_approach'] = decel_app(col[9])

In [12]:
# Change column in boolean type to integer value 0/1
qar_df_2021[['one_engine_taxi_out', 'one_engine_taxi_in','is_less_flaps_takeoff','is_decel_approach']] = qar_df_2021[['one_engine_taxi_out', 'one_engine_taxi_in','is_less_flaps_takeoff','is_decel_approach']].astype(int)

In [13]:
# Count flight that is not departure from VHHH
qar_df_2021['is_not_hkg']=qar_df_2021.departure_aerodrome != 'VHHH'

In [14]:
# Remove TJS data in year 2022

qar_df_2021 = qar_df_2021[~((qar_df_2021.date >= "2022-01-01")&(qar_df_2021.aircraft_registration == "HS-TJS"))]

In [15]:
# data are summary by sum, average and count.

sum_col = [
    'one_engine_taxi_out',
    'is_less_flaps_takeoff',
    'pack_off_takeoff',
    'is_decel_approach',
    'reduced_flaps_landing',
    'idle_reverse_landing_performed',
    'one_engine_taxi_in',
    'apu_used_during_taxi_in',
    'is_not_hkg'
]
avg_col = [
    'acceleration_height',
    'climb_sd',
    'descent_sd',
]

In [16]:
# Create group by object
groupped_df = qar_df_2021.groupby([qar_df_2021.index.year, qar_df_2021.index.month,qar_df_2021.aircraft_type])

In [17]:
summary_by_sum = groupped_df.sum()[sum_col]
summary_by_avg = groupped_df.mean()[avg_col]
total_qar = groupped_df['aircraft_registration'].count()

In [18]:
# Merge all table and rename count of aircraft registration to total_qar
report = summary_by_sum.join(summary_by_avg)
report = report.join(total_qar).rename(columns={'aircraft_registration':'total_qar'})


In [19]:
#rearrange column to fit report
column_order = [
    'total_qar',
    'is_not_hkg',
    'one_engine_taxi_out',
    'is_less_flaps_takeoff',
    'pack_off_takeoff',
    'acceleration_height',
    'climb_sd',
    'descent_sd',
    'is_decel_approach',
    'reduced_flaps_landing',
    'idle_reverse_landing_performed',
    'one_engine_taxi_in',
    'apu_used_during_taxi_in'
]
report = report[column_order]

In [20]:
#change column name
rename_dict = {
    'total_qar' : 'Total QAR',
    'is_not_hkg' : 'Total Flight exc HKG',
    'one_engine_taxi_out' : 'RETO',
    'is_less_flaps_takeoff' : 'Take off flaps',
    'pack_off_takeoff' : 'Pack Off',
    'acceleration_height' : 'Accel Height',
    'climb_sd' : 'CCO',
    'descent_sd' : 'CDO',
    'is_decel_approach' : 'Decel Appch',
    'reduced_flaps_landing' : 'Landing Flaps',
    'idle_reverse_landing_performed' : 'Idle Rev',
    'one_engine_taxi_in' : 'RETI',
    'apu_used_during_taxi_in' : 'APU start'
}
report.rename(columns= rename_dict, inplace=True)

In [21]:
#Export to csv
report.to_csv('C:/Users/peaju/THAI AIRWAYS INTERNATIONAL PUBLIC CO.,LTD/DP - PC - Team 1 - Documents/PC - Team 1/2022/Data/qar_summary.csv')

In [22]:
report.loc[2022][['Total QAR', 'Take off flaps', "RETI"]]

Total QAR  Take off flaps  RETI
date aircraft_type                                 
1    A359                 354             342   177
     B77E                  40              40    30
     B77W                 588             422   316
     B788                  67              64     5
2    A359                 317             313   134
     B77E                  63              59    27
     B77W                 543             383   301
     B788                  31              31     0
3    A359                 485             469   205
     B77E                 108             106    72
     B77W                 640             435   394
     B788                 113             111     3
     B789                   4               3     0
4    A359                 574             559   209
     B77E                  85              84    59
     B77W                 685             446   368
     B788                 269             265    26
     B789                  18              17     0
5    A359                 618             605   240
     B77E                 108             106    74
     B77W                 652             435   380
     B788                 124             123     2
     B789                  32              31     1
6    A359                  56              56    30
     B77W                  59              45    30

In [23]:
report

Total QAR  Total Flight exc HKG  RETO  \
date date aircraft_type                                          
2021 1    A359                  88                    84    15   
          B77E                  45                    45     0   
          B77W                  97                    91     3   
     2    A359                 162                   147    36   
          B77E                  42                    42     1   
...                            ...                   ...   ...   
2022 5    B788                 124                   124     0   
          B789                  32                    32     0   
     6    A359                  56                    54    14   
          B77W                  59                    58     8   
2080 10   A359                   2                     2     0   

                         Take off flaps  Pack Off  Accel Height  CCO   CDO  \
date date aircraft_type                                                      
2021 1    A359                       87     83.00      2,531.77 1.50  7.00   
          B77E                       43      4.00      2,633.36 1.36  8.21   
          B77W                       48      3.00      2,885.84 2.58  8.60   
     2    A359                      157    156.00      2,309.46 1.72  6.83   
          B77E                       40      1.00      2,804.79 1.04  7.37   
...                                 ...       ...           ...  ...   ...   
2022 5    B788                      123      0.00      2,901.97 1.56  8.96   
          B789                       31      1.00      3,188.34 0.99 10.08   
     6    A359                       56     56.00      2,478.29 2.13  7.17   
          B77W                       45      7.00      2,526.75 1.89  9.69   
2080 10   A359                        2      2.00      1,014.00 2.06  7.63   

                         Decel Appch  Landing Flaps  Idle Rev  RETI  APU start  
date date aircraft_type                                                         
2021 1    A359                    87          83.00     87.00    29       0.00  
          B77E                    43          26.00     37.00     8      34.00  
          B77W                    94          38.00     77.00    23      78.00  
     2    A359                   156         159.00    161.00    62       0.00  
          B77E                    42          23.00     40.00    20      31.00  
...                              ...            ...       ...   ...        ...  
2022 5    B788                   117          70.00     69.00     2     123.00  
          B789                    31          11.00     11.00     1      31.00  
     6    A359                    52          53.00     55.00    30       0.00  
          B77W                    55          25.00     48.00    30      58.00  
2080 10   A359                     2           2.00      2.00     0       0.00  

[70 rows x 13 columns]